In [81]:

import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

import sys

sys.path.append("/Users/antoinerazeghi/Documents/Projects/AirLiquide/airliquide/")

import hub_clustering.params as p
import hub_clustering.utils as f
import hub_clustering.hubstation_sizer as sizer

scenario = "optimistic"


dataset = f.load_data()
df = f.filter_dataset(dataset, scenario=scenario)
final_dataset, centroids_df = f.run_kmeans(df)

centroids_df.reset_index(inplace=True)
centroids_df.columns = ["centroid_id", "region", "latitude", "longitude"]
centroids_df.sort_values(by=["region", "centroid_id"])

h2_kg_demand = (
    df["Surface totale"]
    .pipe(sizer.compute_total_volume)
    .pipe(sizer.compute_daily_stock_volume_gone)
    .pipe(sizer.compute_daily_nb_trucks)
    .pipe(sizer.compute_daily_nb_h2_trucks)
    .pipe(sizer.compute_daily_h2_demand)
)

final_dataset["h2_kg_demand"] = h2_kg_demand

df_stations_type = final_dataset.groupby(
    ["Région d'implantation", "centroid"], as_index=False
).agg({"centroid_coord": "first", "h2_kg_demand": "sum"})

df_stations_type["stations_type"] = df_stations_type["h2_kg_demand"].apply(
    sizer.stations_type
)


In [85]:
centroids_df

,centroid_id,region,latitude,longitude
0,0,52,48.110725,-0.711265
1,1,52,47.273352,-2.213891
2,2,52,47.311116,-1.115343
3,3,52,46.998497,-0.868969
4,4,52,46.777473,-1.345666
...,...,...,...,...
178,178,27,46.661640,4.853065
179,179,27,47.749799,3.666738
180,180,27,45.195018,5.686962
181,181,27,47.231824,5.018804


In [89]:
df_stations_type = df_stations_type[["h2_kg_demand", "stations_type"]].merge(centroids_df, left_index=True, right_on="centroid_id")

In [90]:
df_stations_type["station_utilization_rate"] = df_stations_type.apply(
    lambda x: sizer.station_utilization_rate(x.h2_kg_demand, x.stations_type), axis=1
)

fig = sizer.visualize_station_types_on_map(df_stations_type)

In [91]:
fig.show()

In [93]:
df_stations_type.stations_type.value_counts()["large"]

15